# Test nombre lignes siretise

Copy paste from Coda to fill the information

## Objective(s)

- Lors de [l’US 7: Test nombre lignes siretise avec nouvelles regles de gestion](https://coda.io/d/CreditAgricole_dCtnoqIftTn/US-07-ETS-version-3_su0VF), nous avons créé une table avec l’ensemble des possibilités de tests, trié par ordre de préférence. 
- Lors de l’US, [Creation table distance word2vec et merge table ets inpi insee cas](https://coda.io/d/CreditAgricole_dCtnoqIftTn/US-07-ETS-version-3_su0VF), nous avons crée deux variables pour les tests, a savoir le test sur la distance de cosine, et le test de levhenstein. 
  - Afin de séparer les doublons, il suffit de récupérer le rank minimum par index. Celui ci va nous donner le meilleur des probables. 
  - Il est bien sur possible d’avoir encore des doublons, dans ces cas la, il faut aller plus loin dans la rédaction des tests
  - L’objectif de cette US est de récupérer le rank minimum de la table ets_inpi_insee_cases puis de faire une analyse brève des index récupérés. 

## Metadata 

- - Metadata parameters are available here: [Ressources_suDYJ#_luZqd](http://Ressources_suDYJ#_luZqd)

  - Task type:

    - Jupyter Notebook

  - Users: :

    - [Thomas Pernet](mailto:t.pernetcoudrier@gmail.com)

  - Watchers:

    - [Thomas Pernet](mailto:t.pernetcoudrier@gmail.com)

  - Estimated Log points:

    - One being a simple task, 15 a very difficult one
        -  8

  - Task tag

    -  \#sql-query,#matching,#similarite,#regle-de-gestion

  - Toggl Tag

    - \#data-analysis
  
## Input Cloud Storage [AWS/GCP]

If link from the internet, save it to the cloud first

### Tables [AWS/BigQuery]

- Batch 1:

  - Select Provider: Athena

    -  Select table(s): ets_inpi_insee_cases_distance

    - Select only tables created from the same notebook, else copy/paste selection to add new input tables

      - If table(s) does not exist, add them: 

        Add New Table

      - Information:

      - Region: 

        - NameEurope (Paris)
          - Code: eu-west-3

        - Database: inpi

        - Notebook construction file: [07_bis_creation_distance](https://github.com/thomaspernet/InseeInpi_matching/blob/master/Notebooks_matching/Data_preprocessed/programme_matching/02_siretisation/07_bis_creation_distance.md)
    
## Destination Output/Delivery

- AWS

    -  Athena: 

      -  Region: Europe (Paris)
        - Database: inpi
        - Tables (Add name new table): ets_inpi_insee_cases_rank
        - List new tables
        - ets_inpi_insee_cases_rank 

## Things to know (Steps, Attention points or new flow of information)

### Sources of information  (meeting notes, Documentation, Query, URL)


## Connexion serveur

In [1]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_athena import service_athena
from awsPy.aws_s3 import service_s3
from pathlib import Path
import pandas as pd
import numpy as np
import os, shutil
bucket = 'calfdata'
path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = r"{}/credential_AWS.json".format(parent_path)

region = 'eu-west-3'
bucket = 'calfdata'

In [2]:
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = region)
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = bucket, verbose = False) 
#athena = service_athena.connect_athena(client = client,
#                      bucket = bucket) 
s3_output = 'INPI/sql_output'

In [18]:
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)
pd.set_option('display.max_columns', None)

# Creation tables

## Steps

- Recupération du rank minimum par index
- Filtre les lignes correspondant au rank minimum par index

In [3]:
query = """
DROP TABLE `inpi.ets_inpi_insee_cases_rank`;
"""
s3.run_query(
        query=query,
        database='inpi',
        s3_output='INPI/sql_output'
    )

Execution ID: 45136c83-bf59-43be-9012-20e233d6fc12


{'QueryExecutionId': '45136c83-bf59-43be-9012-20e233d6fc12',
 'ResponseMetadata': {'RequestId': 'cb873d36-2dbd-4bff-b772-4375e55eeb6e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 29 Aug 2020 05:29:34 GMT',
   'x-amzn-requestid': 'cb873d36-2dbd-4bff-b772-4375e55eeb6e',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [4]:
query = """
CREATE TABLE inpi.ets_inpi_insee_cases_rank
WITH (
  format='PARQUET'
) AS
WITH tb_min_rank AS (
SELECT 
min_rank,
  row_id, 
  ets_inpi_insee_cases_distance.index_id, 
  sequence_id, 
  siren, 
  siret,
  list_inpi, 
  lenght_list_inpi, 
  list_insee, 
  lenght_list_insee, 
  inpi_except, 
  insee_except, 
  intersection, 
  union_, 
  pct_intersection, 
  len_inpi_except, 
  len_insee_except, 
  status_cas,
  unzip_inpi,
  unzip_insee,
  max_cosine_distance,
  test_distance_cosine,
  levenshtein_distance,
  test_distance_levhenstein, 
  count_initial_insee, 
  count_inpi_siren_siret, 
  count_inpi_siren_sequence, 
  count_inpi_sequence_siret, 
  count_inpi_sequence_stat_cas_siret, 
  count_inpi_index_id_siret, 
  count_inpi_index_id_stat_cas_siret, 
  count_inpi_index_id_stat_cas, 
  index_id_duplicate, 
  test_sequence_siret, 
  test_index_siret, 
  test_siren_insee_siren_inpi, 
  test_sequence_siret_many_cas, 
  list_numero_voie_matching_inpi, 
  list_numero_voie_matching_insee, 
  intersection_numero_voie, 
  union_numero_voie, 
  test_list_num_voie, 
  datecreationetablissement, 
  date_debut_activite, 
  test_date, 
  etatadministratifetablissement, 
  status_admin, 
  test_status_admin, 
  etablissementsiege, 
  status_ets, 
  test_siege, 
  codecommuneetablissement, 
  code_commune, 
  test_code_commune, 
  codepostaletablissement, 
  code_postal_matching, 
  numerovoieetablissement, 
  numero_voie_matching, 
  test_numero_voie, 
  typevoieetablissement, 
  type_voie_matching, 
  test_type_voie, 
  test_adresse_cas_1_3_4, 
  index_id_dup_has_cas_1_3_4, 
  test_duplicates_is_in_cas_1_3_4, 
  enseigne, 
  enseigne1etablissement, 
  enseigne2etablissement, 
  enseigne3etablissement, 
  test_enseigne,
  key_except_to_test
FROM ets_inpi_insee_cases_distance 
INNER JOIN (
  SELECT index_id, MIN(rank) AS min_rank
FROM ets_inpi_insee_cases_distance
GROUP BY index_id
  ) as tb_min_rank
ON ets_inpi_insee_cases_distance.index_id = tb_min_rank.index_id AND
ets_inpi_insee_cases_distance.rank = tb_min_rank.min_rank
  ) 
  SELECT 
  min_rank,
  row_id, 
  tb_min_rank.index_id, 
  count_index,
  sequence_id, 
  siren, 
  siret,
  list_inpi, 
  lenght_list_inpi, 
  list_insee, 
  lenght_list_insee, 
  inpi_except, 
  insee_except, 
  intersection, 
  union_, 
  pct_intersection, 
  len_inpi_except, 
  len_insee_except, 
  status_cas,
  unzip_inpi,
  unzip_insee,
  max_cosine_distance,
  test_distance_cosine,
  levenshtein_distance,
  test_distance_levhenstein, 
  count_initial_insee, 
  count_inpi_siren_siret, 
  count_inpi_siren_sequence, 
  count_inpi_sequence_siret, 
  count_inpi_sequence_stat_cas_siret, 
  count_inpi_index_id_siret, 
  count_inpi_index_id_stat_cas_siret, 
  count_inpi_index_id_stat_cas, 
  index_id_duplicate, 
  test_sequence_siret, 
  test_index_siret, 
  test_siren_insee_siren_inpi, 
  test_sequence_siret_many_cas, 
  list_numero_voie_matching_inpi, 
  list_numero_voie_matching_insee, 
  intersection_numero_voie, 
  union_numero_voie, 
  test_list_num_voie, 
  datecreationetablissement, 
  date_debut_activite, 
  test_date, 
  etatadministratifetablissement, 
  status_admin, 
  test_status_admin, 
  etablissementsiege, 
  status_ets, 
  test_siege, 
  codecommuneetablissement, 
  code_commune, 
  test_code_commune, 
  codepostaletablissement, 
  code_postal_matching, 
  numerovoieetablissement, 
  numero_voie_matching, 
  test_numero_voie, 
  typevoieetablissement, 
  type_voie_matching, 
  test_type_voie, 
  test_adresse_cas_1_3_4, 
  index_id_dup_has_cas_1_3_4, 
  test_duplicates_is_in_cas_1_3_4, 
  enseigne, 
  enseigne1etablissement, 
  enseigne2etablissement, 
  enseigne3etablissement, 
  test_enseigne,
  key_except_to_test 
  FROM tb_min_rank
  LEFT JOIN (
    SELECT index_id, COUNT(*) AS count_index
    FROM tb_min_rank
    GROUP BY index_id
    ) as tb_nb_index
    ON tb_min_rank.index_id = tb_nb_index.index_id
"""

output = s3.run_query(
            query=query,
            database='inpi',
            s3_output=s3_output,
  filename = None, ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

Execution ID: db96e130-6aa0-4ae0-b963-bba236f630ae


# Analyse

## Count nombre lignes  & index

Le nombre de lignes est de:

In [5]:
query = """
SELECT COUNT(*)
FROM ets_inpi_insee_cases_rank 
"""

output = s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'cnt_nb_lignes_rank', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )
output

Execution ID: ae1ec9db-ae19-40da-9c97-7903e84e34b5


,_col0
0,9450637


Le nombre d'index est de 

In [6]:
query = """
SELECT COUNT(distinct(index_id))
FROM ets_inpi_insee_cases_rank 
"""

output = s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'cnt_nb_index_rank', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )
output

Execution ID: be591202-ebc5-4028-91d4-06a60d9872a1


,_col0
0,9421163


## Evaluation des doublons

In [7]:
query = """
SELECT count_index, COUNT(*) as ligne_dup
FROM ets_inpi_insee_cases_rank 
GROUP BY count_index 
ORDER BY count_index
"""

nb_ligne = s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'cnt_nb_dup_lignes_rank', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
)

Execution ID: 4dd63b2f-db14-4711-b2b3-1927735297a8


In [8]:
query = """
SELECT count_index, COUNT(DISTINCT(index_id)) as index_dup
FROM ets_inpi_insee_cases_rank 
GROUP BY count_index 
ORDER BY count_index
"""

nb_index = s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'cnt_nb_dup_index_rank', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )

Execution ID: 153b7586-dada-4db8-82c4-eeae5ff65378


In [9]:
(
pd.concat([    
 pd.concat([
    pd.concat(
    [
        nb_ligne.sum().to_frame().T.rename(index = {0:'total'}), 
        nb_ligne
    ], axis = 0),
    ],axis = 1,keys=["Lignes"]),
    (
 pd.concat([
    pd.concat(
    [
        nb_index.sum().to_frame().T.rename(index = {0:'total'}), 
        nb_index
    ], axis = 0),
    ],axis = 1,keys=["Index"])
)],axis= 1
    )
    .style
    .format("{:,.0f}")
                  .bar(subset= [
                      ('Lignes','ligne_dup'),
                      ('Index','index_dup'),
                      
                  ],
                       color='#d65f5f')
)

Nombre d'index récuperé

In [27]:
9397225

9397225

Nombre d'index a trouver

In [28]:
9421163

9421163

Pourcentage de probable trouvé

In [30]:
round(9397225 / 9421163, 4)

0.9975

## Analyse des ranks

In [17]:
pd.set_option('display.max_colwidth', None)

In [12]:
query = """
SELECT 
count_index, 
  approx_percentile(min_rank, ARRAY[.1, .15, .20, 0.25,0.50,0.75,.80,.85,.86,.87, .88, .89,.90,0.95, 0.99]) as pct_min_rank
FROM 
  ets_inpi_insee_cases_rank
GROUP BY count_index  
ORDER BY count_index
"""
s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'distribution_rank', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )

Execution ID: 2373121f-cb23-44bf-817b-c2e62acab2ec


,count_index,pct_min_rank
0,1,"[3019, 3021, 3021, 3021, 3451, 4315, 8851, 15007, 15496, 15659, 16511, 20475, 20493, 21463, 32767]"
1,2,"[3021, 3345, 3345, 3345, 3451, 18007, 18391, 19016, 19031, 19071, 20991, 23408, 23875, 24865, 30683]"
2,3,"[1561, 6426, 13227, 17989, 19015, 24363, 24793, 24848, 24859, 24865, 24867, 24869, 24905, 30213, 30699]"
3,4,"[18061, 19009, 19037, 20476, 20812, 24811, 25230, 26737, 26794, 27135, 27281, 27781, 29711, 30701, 32961]"
4,5,"[1512, 1620, 7398, 17990, 18549, 19342, 23826, 24793, 24847, 24847, 24869, 24869, 29653, 30673, 30683]"
5,6,"[18480, 18529, 18534, 18587, 23880, 24867, 24895, 30216, 30216, 30629, 30679, 30679, 30679, 30701, 30753]"
6,7,"[593, 593, 593, 13229, 19015, 24851, 24851, 24865, 24865, 24865, 24865, 24869, 24869, 24869, 24869]"
7,8,"[5989, 5989, 5989, 17557, 18064, 19038, 19038, 23394, 23394, 23394, 23394, 24847, 24847, 30194, 30194]"
8,9,"[18060, 18065, 18066, 19009, 24847, 30679, 30682, 30683, 30683, 30683, 30683, 30683, 30737, 30737, 30737]"
9,10,"[19015, 19015, 19015, 19015, 19015, 19015, 19015, 19015, 19015, 19015, 19015, 19015, 19015, 19015, 19015]"


Prenons par exemple, le rank 32897, qui correspond a la règle suivante:

In [15]:
query ="""
SELECT *
FROM regles_tests 
WHERE rank = 32897
"""
s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'rules', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )

Execution ID: 755eb8f0-d8a9-4d5d-a88c-c08b0601913d


,status_cas,index_id_duplicate,test_list_num_voie,test_siren_insee_siren_inpi,test_siege,test_enseigne,test_distance_cosine,test_distance_levhenstein,test_date,test_status_admin,rank
0,CAS_6,False,True,False,False,True,True,False,False,True,32897


Ci dessous, un ensemble de lignes correspondant a la règle 32897

In [19]:
query ="""
SELECT * 
FROM ets_inpi_insee_cases_rank 
WHERE min_rank = 32897
LIMIT 10
"""
s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'rules_32897', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )

Execution ID: 000a9117-32bb-40b9-9679-caa29d1963ee


,min_rank,row_id,index_id,count_index,sequence_id,siren,siret,list_inpi,lenght_list_inpi,list_insee,lenght_list_insee,inpi_except,insee_except,intersection,union_,pct_intersection,len_inpi_except,len_insee_except,status_cas,unzip_inpi,unzip_insee,max_cosine_distance,test_distance_cosine,levenshtein_distance,test_distance_levhenstein,count_initial_insee,count_inpi_siren_siret,count_inpi_siren_sequence,count_inpi_sequence_siret,count_inpi_sequence_stat_cas_siret,count_inpi_index_id_siret,count_inpi_index_id_stat_cas_siret,count_inpi_index_id_stat_cas,index_id_duplicate,test_sequence_siret,test_index_siret,test_siren_insee_siren_inpi,test_sequence_siret_many_cas,list_numero_voie_matching_inpi,list_numero_voie_matching_insee,intersection_numero_voie,union_numero_voie,test_list_num_voie,datecreationetablissement,date_debut_activite,test_date,etatadministratifetablissement,status_admin,test_status_admin,etablissementsiege,status_ets,test_siege,codecommuneetablissement,code_commune,test_code_commune,codepostaletablissement,code_postal_matching,numerovoieetablissement,numero_voie_matching,test_numero_voie,typevoieetablissement,type_voie_matching,test_type_voie,test_adresse_cas_1_3_4,index_id_dup_has_cas_1_3_4,test_duplicates_is_in_cas_1_3_4,enseigne,enseigne1etablissement,enseigne2etablissement,enseigne3etablissement,test_enseigne,key_except_to_test
0,32897,2634015,5840905,1,2611852,434367884,43436788400021,"[ROUTE, NAPOLLON, RN]",3,"[ROUTE, NATIONALE, RTE, NAPOLLON]",4,[RN],"[NATIONALE, RTE]",2.0,5.0,0.40,1,2,CAS_6,RN,NATIONALE,0.710957,True,8,False,3,1,2,1,1,1,1,1,False,True,True,False,True,[96],[96],1.0,1.0,True,2008-03-01 00:00:00.000,2001-01-15 00:00:00.000,False,A,A,True,True,False,False,13005,13005,True,13400,13400,NaN,96,NaN,RTE,RTE,True,False,False,NaN,NEGO'S AUTOS,NEGO'S AUTOS,NaN,NaN,True,"[{field0=RN, field1=[NATIONALE, RTE]}]"
1,32897,1210235,1413347,1,507292,331902064,33190206400020,"[AVENUE, EME, RI]",3,"[AVENUE, EME, REG, INFANTERIE]",4,[RI],"[REG, INFANTERIE]",2.0,5.0,0.40,1,2,CAS_6,RI,INFANTERIE,0.638259,True,8,False,2,1,2,1,1,1,2,1,False,True,False,False,True,"[70, 11]","[70, 11]",2.0,2.0,True,2005-07-18 00:00:00.000,1985-04-01 00:00:00.000,False,A,A,True,True,False,False,82121,82121,True,82000,82000,70.0,70,True,AV,AV,True,False,False,NaN,PHARMACIE DU 11EME,PHARMACIE DU 11EME,NaN,NaN,True,"[{field0=RI, field1=[REG, INFANTERIE]}]"
2,32897,4121777,6344320,1,5230804,529123374,52912337400033,"[RUE, HENRI, ANTOINE, MAURRAS, IMM, NEREIS, ZI, SAUMATY, SEON]",9,"[RUE, HENRI, ANTOINE, MAURRAS, IMMEUBLE, NEREIS, Z, I, SAUMATY, SEON]",10,"[IMM, ZI]","[IMMEUBLE, Z, I]",7.0,12.0,0.58,2,3,CAS_6,IMM,IMMEUBLE,0.818024,True,5,False,4,3,4,1,1,1,1,1,False,True,True,False,True,[4],[4],1.0,1.0,True,2011-12-01 00:00:00.000,2010-12-01 00:00:00.000,False,A,A,True,True,False,False,13216,13216,True,13016,13016,4.0,4,True,RUE,RUE,True,False,False,NaN,P.I.B SOLUTIONS,P.I.B SOLUTIONS,NaN,NaN,True,"[{field0=IMM, field1=[IMMEUBLE, Z, I]}, {field0=ZI, field1=[IMMEUBLE, Z, I]}]"
3,32897,4528485,5920480,1,3073440,448334144,44833414400020,"[RUE, FER, A, CHEVAL, BATIMENT, B, ZI, MIRANDE]",8,"[RUE, FER, A, CHEVAL, BAT, B, Z, I, MIRANDE]",9,"[BATIMENT, ZI]","[BAT, Z, I]",6.0,11.0,0.55,2,3,CAS_6,BATIMENT,BAT,0.732788,True,5,False,2,1,2,1,1,1,1,1,False,True,True,False,True,[3],[3],1.0,1.0,True,2013-02-06 00:00:00.000,2003-05-02 00:00:00.000,False,A,A,True,True,False,False,66172,66172,True,66240,66240,3.0,3,True,RUE,RUE,True,False,False,NaN,SANI-CLIM,SANI-CLIM,NaN,NaN,True,"[{field0=BATIMENT, field1=[BAT, Z, I]}, {field0=ZI, field1=[BAT, Z, I]}]"
4,32897,8469463,6090798,1,3840118,489386755,48938675500024,"[CENTRE, COMMERCIAL, RN]",3,"[ROUTE, NATIONALE, CENTRE, CCIAL]",4,"[COMMERCIAL, RN]","[ROUTE, NATIONALE, CCIAL]",1.0,6.0,0.17,2,3,CAS_6,COMMERCIAL,CCIAL,0.777006,True,5,False,2,1,2,1,1,1,1,1,False,True,True,False,True,[20],[20],1.0,1.0,True,1900-01-01 00:00:00.000,2006-08-08 00:00:00.000,False,A,A,True,True,False,False,91665,91665,True,91620,91620,NaN,20

## Analyse des cas pour index unique

In [20]:
query = """
SELECT status_cas, count(*) as nb_unique
FROM ets_inpi_insee_cases_rank 
WHERE count_index = 1
GROUP BY status_cas
ORDER BY status_cas
"""

tb = s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'nb_cas_index_unique_rank', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )
(
    tb.
assign(
cumsum = lambda x: x['nb_unique'].cumsum(),
    pct_total = lambda x: x['nb_unique']/x['nb_unique'].sum(),
    pct_total_cum = lambda x: x['pct_total'].cumsum()
)
    .style
    .format("{0:,.2%}", subset=["pct_total", 'pct_total_cum'])
    .bar(subset= ['cumsum'],
                       color='#d65f5f')
    .bar(subset= ['nb_unique'],
                       color='#228B22')
)

Execution ID: d3a20a3f-27db-4a18-97d8-bc1ecd720072


,status_cas,nb_unique,cumsum,pct_total,pct_total_cum
0,CAS_1,7496562,7496562,79.77%,79.77%
1,CAS_3,326518,7823080,3.47%,83.25%
2,CAS_4,458261,8281341,4.88%,88.13%
3,CAS_5,765832,9047173,8.15%,96.27%
4,CAS_6,139140,9186313,1.48%,97.76%
5,CAS_7,210912,9397225,2.24%,100.00%


## Exemple doublons

Il reste encore des doublons. C'est le cas lorsque le test sur la distance retourne le même resultat pour toutes les lignes. 

Une des possibilités serait de récupérer la valeur la plus élevée de `pct_intersection`. 

In [21]:
query ="""
SELECT * 
FROM ets_inpi_insee_cases_rank 
WHERE count_index = 10
ORDER BY min_rank
LIMIT 10
"""
s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'count_index_10', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )

Execution ID: d90ff6ee-77a5-444c-9b8e-1bfcd66c0766


,min_rank,row_id,index_id,count_index,sequence_id,siren,siret,list_inpi,lenght_list_inpi,list_insee,lenght_list_insee,inpi_except,insee_except,intersection,union_,pct_intersection,len_inpi_except,len_insee_except,status_cas,unzip_inpi,unzip_insee,max_cosine_distance,test_distance_cosine,levenshtein_distance,test_distance_levhenstein,count_initial_insee,count_inpi_siren_siret,count_inpi_siren_sequence,count_inpi_sequence_siret,count_inpi_sequence_stat_cas_siret,count_inpi_index_id_siret,count_inpi_index_id_stat_cas_siret,count_inpi_index_id_stat_cas,index_id_duplicate,test_sequence_siret,test_index_siret,test_siren_insee_siren_inpi,test_sequence_siret_many_cas,list_numero_voie_matching_inpi,list_numero_voie_matching_insee,intersection_numero_voie,union_numero_voie,test_list_num_voie,datecreationetablissement,date_debut_activite,test_date,etatadministratifetablissement,status_admin,test_status_admin,etablissementsiege,status_ets,test_siege,codecommuneetablissement,code_commune,test_code_commune,codepostaletablissement,code_postal_matching,numerovoieetablissement,numero_voie_matching,test_numero_voie,typevoieetablissement,type_voie_matching,test_type_voie,test_adresse_cas_1_3_4,index_id_dup_has_cas_1_3_4,test_duplicates_is_in_cas_1_3_4,enseigne,enseigne1etablissement,enseigne2etablissement,enseigne3etablissement,test_enseigne,key_except_to_test
0,19015,6316961,6894126,10,5086610,523631497,52363149700261,"[RUE, D, ATHENES, MAT, J, AMADE]",6,"[RUE, FRERES, ROSNY, ECOLE, D, ALEMBERT]",6,"[ATHENES, MAT, J, AMADE]","[FRERES, ROSNY, ECOLE, ALEMBERT]",2.0,10.0,0.20,4,4,CAS_5,MAT,GS,0.700185,True,3,False,33,28,32,17,5,17,5,3,True,False,False,False,False,NaN,NaN,NaN,NaN,NaN,2012-03-12 00:00:00.000,2012-03-12 00:00:00.000,True,A,A,True,False,False,True,66136,66136,True,66000,66000,NaN,NaN,NaN,RUE,RUE,True,False,False,TO_FIND,NaN,NaN,NaN,NaN,NaN,"[{field0=ATHENES, field1=[FRERES, ROSNY, ECOLE, ALEMBERT]}, {field0=MAT, field1=[FRERES, ROSNY, ECOLE, ALEMBERT]}, {field0=J, field1=[FRERES, ROSNY, ECOLE, ALEMBERT]}, {field0=AMADE, field1=[FRERES, ROSNY, ECOLE, ALEMBERT]}]"
1,19015,6316961,6894126,10,5086610,523631497,52363149700261,"[RUE, D, ATHENES, MAT, J, AMADE]",6,"[RUE, FRERES, ROSNY, ECOLE, D, ALEMBERT]",6,"[ATHENES, MAT, J, AMADE]","[FRERES, ROSNY, ECOLE, ALEMBERT]",2.0,10.0,0.20,4,4,CAS_5,MAT,GS,0.700185,True,3,False,33,28,32,17,5,17,5,3,True,False,False,False,False,NaN,NaN,NaN,NaN,NaN,2012-03-12 00:00:00.000,2012-03-12 00:00:00.000,True,A,A,True,False,False,True,66136,66136,True,66000,66000,NaN,NaN,NaN,RUE,RUE,True,False,False,TO_FIND,NaN,NaN,NaN,NaN,NaN,"[{field0=ATHENES, field1=[FRERES, ROSNY, ECOLE, ALEMBERT]}, {field0=MAT, field1=[FRERES, ROSNY, ECOLE, ALEMBERT]}, {field0=J, field1=[FRERES, ROSNY, ECOLE, ALEMBERT]}, {field0=AMADE, field1=[FRERES, ROSNY, ECOLE, ALEMBERT]}]"
2,19015,6316960,6894126,10,5086610,523631497,52363149700279,"[RUE, D, ATHENES, MAT, J, AMADE]",6,"[RUE, PAUL, VALERY, ECOLE, PIERRE, COUBERTIN]",6,"[D, ATHENES, MAT, J, AMADE]","[PAUL, VALERY, ECOLE, PIERRE, COUBERTIN]",1.0,11.0,0.09,5,5,CAS_5,MAT,GS,0.700185,True,3,False,33,28,32,17,5,17,5,3,True,False,False,False,False,NaN,NaN,NaN,NaN,NaN,2012-03-12 00:00:00.000,2012-03-12 00:00:00.000,True,A,A,True,False,False,True,66136,66136,True,66000,66000,NaN,NaN,NaN,RUE,RUE,True,False,False,TO_FIND,NaN,NaN,NaN,NaN,NaN,"[{field0=D, field1=[PAUL, VALERY, ECOLE, PIERRE, COUBERTIN]}, {field0=ATHENES, field1=[PAUL, VALERY, ECOLE, PIERRE, COUBERTIN]}, {field0=MAT, field1=[PAUL, VALERY, ECOLE, PIERRE, COUBERTIN]}, {field0=J, field1=[PAUL, VALERY, ECOLE, PIERRE, COUBERTIN]}, {field0=AMADE, field1=[PAUL, VALERY, ECOLE, PIERRE, COUBERTIN]}]"
3,19015,6316960,6894126,10,5086610,523631497,52363149700279,"[RUE, D, ATHENES, MAT, J, AMADE]",6,"[RUE, PAUL, VALERY, ECOLE, PIERRE, COUBERTIN]",6,"[D, ATHENES, MAT, J, AMADE]","[PAUL, VALERY, ECOLE, PIERRE, COUBERTIN]",1.0,11.0,0.09,5,5,CAS_5,MAT,GS,0.700185,True,3,False,33,28,32,17,5,17,5,3,True,False,False,False,False,NaN,NaN,NaN,NaN,NaN,201

D'autres possibilités de doublons sont dues a des erreurs de préparation de la donnée. 

In [22]:
query = """
SELECT * 
FROM ets_inpi_insee_cases_rank 
WHERE count_index = 2
ORDER BY index_id, min_rank
LIMIT 10
"""
s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'count_index_2', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )

Execution ID: 8e253bc7-3fdc-4115-9827-5b32be40ea82


,min_rank,row_id,index_id,count_index,sequence_id,siren,siret,list_inpi,lenght_list_inpi,list_insee,lenght_list_insee,inpi_except,insee_except,intersection,union_,pct_intersection,len_inpi_except,len_insee_except,status_cas,unzip_inpi,unzip_insee,max_cosine_distance,test_distance_cosine,levenshtein_distance,test_distance_levhenstein,count_initial_insee,count_inpi_siren_siret,count_inpi_siren_sequence,count_inpi_sequence_siret,count_inpi_sequence_stat_cas_siret,count_inpi_index_id_siret,count_inpi_index_id_stat_cas_siret,count_inpi_index_id_stat_cas,index_id_duplicate,test_sequence_siret,test_index_siret,test_siren_insee_siren_inpi,test_sequence_siret_many_cas,list_numero_voie_matching_inpi,list_numero_voie_matching_insee,intersection_numero_voie,union_numero_voie,test_list_num_voie,datecreationetablissement,date_debut_activite,test_date,etatadministratifetablissement,status_admin,test_status_admin,etablissementsiege,status_ets,test_siege,codecommuneetablissement,code_commune,test_code_commune,codepostaletablissement,code_postal_matching,numerovoieetablissement,numero_voie_matching,test_numero_voie,typevoieetablissement,type_voie_matching,test_type_voie,test_adresse_cas_1_3_4,index_id_dup_has_cas_1_3_4,test_duplicates_is_in_cas_1_3_4,enseigne,enseigne1etablissement,enseigne2etablissement,enseigne3etablissement,test_enseigne,key_except_to_test
0,1075,928046,434,2,9015729,844749614,84474961400021,"[RUE, NOEL, MACHET]",3,"[RUE, NOEL, MACHET]",3,NaN,NaN,3.0,3.0,1.00,0,0,CAS_1,NaN,NaN,NaN,NaN,NaN,NaN,3,3,2,2,2,2,2,1,True,False,False,True,True,NaN,NaN,NaN,NaN,NaN,2018-12-17 00:00:00.000,2018-12-17 00:00:00.000,True,A,A,True,False,False,True,73304,73304,True,73150,73150,NaN,NaN,NaN,RUE,RUE,True,True,True,TO_KEEP,NaN,NaN,NaN,NaN,NaN,NaN
1,1075,928045,434,2,9015729,844749614,84474961400039,"[RUE, NOEL, MACHET]",3,"[RUE, NOEL, MACHET]",3,NaN,NaN,3.0,3.0,1.00,0,0,CAS_1,NaN,NaN,NaN,NaN,NaN,NaN,3,3,2,2,2,2,2,1,True,False,False,True,True,NaN,NaN,NaN,NaN,NaN,2018-12-17 00:00:00.000,2018-12-17 00:00:00.000,True,A,A,True,False,False,True,73304,73304,True,73150,73150,NaN,NaN,NaN,RUE,RUE,True,True,True,TO_KEEP,NaN,NaN,NaN,NaN,NaN,NaN
2,3345,1107792,1265,2,9090868,847918893,84791889300011,"[RUE, PENTHIEVRE]",2,"[RUE, PENTHIEVRE]",2,NaN,NaN,2.0,2.0,1.00,0,0,CAS_1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,2,1,1,1,1,1,False,True,True,True,True,[10],[10],1.0,1.0,True,2019-01-30 00:00:00.000,NaN,NaN,A,A,True,True,False,False,75108,75108,True,75008,75008,10.0,10.0,True,RUE,RUE,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3345,1107791,1265,2,9090868,847918893,84791889300011,"[RUE, PENTHIEVRE]",2,"[RUE, PENTHIEVRE]",2,NaN,NaN,2.0,2.0,1.00,0,0,CAS_1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,2,1,1,1,1,1,False,True,True,True,True,[10],[10],1.0,1.0,True,2019-01-30 00:00:00.000,NaN,NaN,A,A,True,True,False,False,75108,75108,True,75008,75008,10.0,10.0,True,RUE,RUE,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3345,1107790,1266,2,9090868,847918893,84791889300011,"[RUE, PENTHIEVRE]",2,"[RUE, PENTHIEVRE]",2,NaN,NaN,2.0,2.0,1.00,0,0,CAS_1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,2,1,1,1,1,1,False,True,True,True,True,[10],[10],1.0,1.0,True,2019-01-30 00:00:00.000,NaN,NaN,A,A,True,True,False,False,75108,75108,True,75008,75008,10.0,10.0,True,RUE,RUE,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3345,1107789,1266,2,9090868,847918893,84791889300011,"[RUE, PENTHIEVRE]",2,"[RUE, PENTHIEVRE]",2,NaN,NaN,2.0,2.0,1.00,0,0,CAS_1,NaN,NaN,NaN,NaN,NaN,NaN,1,1,2,1,1,1,1,1,False,True,True,True,True,[10],[10],1.0,1.0,True,2019-01-30 00:00:00.000,NaN,NaN,A,A,True,True,False,False,75108,75108,True,75008,75008,10.0,10.0,True,RUE,RUE,True,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,15169,337689,1844,2,8980205,844259390,84425939000376,"[LIEU, DIT, NID, CHIEN, AH, _, CHEMIN, SAINT, OMER]",9,"[CHEMIN, SAINT, OMER, LIEU, DIT, NID, CHIEN, AH]",8,[_],NaN,8.0,9.0,0.89,1,0,CAS_4,NaN,NaN,NaN,NaN,NaN,NaN,51,47,46,1,1,1,1,1,False,True,True,False,True,"[117, 1046]","[1046, 117]",2.0,2.0,True,2019-09-12 00:00:00.000,2019-09-12 00:00:00.000,True,A

C'est le cas pour le siren 847918893. Il y a deux lignes pour la même date de transmission. Il aurait fallu prendre uniquement la dernière ligne.

In [23]:
query = """
SELECT *
FROM ets_final_sql 
WHERE index_id = 1265
"""
s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'issue_siren_847918893', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )

Execution ID: 8f3a3f51-df97-421f-88ca-5d25aefb0300


,index_id,sequence_id,siren,code_greffe,nom_greffe,numero_gestion,id_etablissement,status,origin,date_greffe,file_timestamp,libelle_evt,last_libele_evt,status_admin,type,status_ets,siège_pm,rcs_registre,adresse_ligne1,adresse_ligne2,adresse_ligne3,adresse_reconstituee_inpi,adresse_regex_inpi,adresse_distance_inpi,list_numero_voie_matching_inpi,numero_voie_matching,voie_clean,type_voie_matching,code_postal,code_postal_matching,ville,ville_matching,code_commune,pays,domiciliataire_nom,domiciliataire_siren,domiciliataire_greffe,domiciliataire_complément,siege_domicile_représentant,nom_commercial,enseigne,activité_ambulante,activité_saisonnière,activité_non_sédentaire,date_début_activité,activité,origine_fonds,origine_fonds_info,type_exploitation,csv_source
0,1265,9090868,847918893,7501,Paris,2019B03224,2,NaN,EVT,2019-02-04 00:00:00.000,2019-02-05 21:28:05.000,Modifications relatives à un établissement,Modifications relatives à un établissement,A,PRI,False,France,Paris,10 rue de Penthièvre,NaN,NaN,10 RUE DE PENTHIEVRE,(?:^|(?<= ))(RUE|PENTHIEVRE)(?:(?= )|$),RUE PENTHIEVRE,[10],10,RUE,RUE,75008,75008,Paris,PARIS,75108,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non,Non,Non,30/01/2019,Organisation d'événements (sauf activités réglementées).,Création,NaN,Exploitation directe,7501_408_20190205_212805_9_ets_nouveau_modifie_EVT.csv
1,1265,9090868,847918893,7501,Paris,2019B03224,2,NaN,EVT,2019-02-04 00:00:00.000,2019-02-05 21:28:05.000,Modifications relatives à un établissement,Etablissement ouvert,A,PRI,False,France,Paris,10 rue de Penthièvre,NaN,NaN,10 RUE DE PENTHIEVRE,(?:^|(?<= ))(RUE|PENTHIEVRE)(?:(?= )|$),RUE PENTHIEVRE,[10],10,RUE,RUE,75008,75008,Paris,PARIS,75108,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non,Non,Non,30/01/2019,Organisation d'événements (sauf activités réglementées).,Création,NaN,Exploitation directe,7501_408_20190205_212805_9_ets_nouveau_modifie_EVT.csv


# Generation report

In [24]:
import os, time, shutil, urllib, ipykernel, json
from pathlib import Path
from notebook import notebookapp

In [25]:
def create_report(extension = "html"):
    """
    Create a report from the current notebook and save it in the 
    Report folder (Parent-> child directory)
    
    1. Exctract the current notbook name
    2. Convert the Notebook 
    3. Move the newly created report
    
    Args:
    extension: string. Can be "html", "pdf", "md"
    
    
    """
    
    ### Get notebook name
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[0].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+ \
                                             'api/sessions?token=' + \
                                             srv['token'])
            sessions = json.load(req)
            notebookname = sessions[0]['name']
        except:
            pass  
    
    sep = '.'
    path = os.getcwd()
    #parent_path = str(Path(path).parent)
    
    ### Path report
    #path_report = "{}/Reports".format(parent_path)
    #path_report = "{}/Reports".format(path)
    
    ### Path destination
    name_no_extension = notebookname.split(sep, 1)[0]
    source_to_move = name_no_extension +'.{}'.format(extension)
    dest = os.path.join(path,'Reports', source_to_move)
    
    ### Generate notebook
    os.system('jupyter nbconvert --no-input --to {} {}'.format(
    extension,notebookname))
    
    ### Move notebook to report folder
    #time.sleep(5)
    shutil.move(source_to_move, dest)
    print("Report Available at this adress:\n {}".format(dest))

In [33]:
create_report(extension = "html")

Report Available at this adress:
 /Users/thomas/Google Drive/Projects/GitHub/Repositories/InseeInpi_matching/Notebooks_matching/Data_preprocessed/programme_matching/02_siretisation/Reports/10_rank_technique_siretisation.html
